In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
from IPython.display import HTML
from matplotlib.widgets import Slider
import ipywidgets as widgets

      
# functions that will used to interfere two waves at the second beam splitter
# interfere_amplitudes calcualtes the effective final amplitude given two waves with their amplitudes and phases
# interfere_phases calcualtes the effective final phase given two waves with their amplitudes and phases
def interfere_amplitudes(a1,phi1,a2,phi2):
    return np.sqrt(np.power((a1*np.sin(phi1)+a2*np.sin(phi2)),2)+np.power((a1*np.cos(phi1)+a2*np.cos(phi2)),2))

def interfere_phases(a1,phi1,a2,phi2):
    return np.arctan((a1*np.sin(phi1)+a2*np.sin(phi2))/(a1*np.cos(phi1)+a2*np.cos(phi2)))

def create_figure(a,vg,c,t,ax,width,xglass,eta,k,Amp,t1,t2):
    r1 = np.sqrt(1-t1*t1); # reflectivity of the first beam splitter
    r2 = np.sqrt(1-t2*t2); # reflectivity of the first beam splitter
    x1 = np.linspace(0, 1,100)
    x2 = np.linspace(1, xglass,100)
    x3 = np.linspace(xglass, xglass+width,100)
    x4 = np.linspace(xglass+width,5,100)
    x5 = np.linspace(1,5,100)
    x6 = np.linspace(5,9,100)
    y1 = np.linspace(0,4,100)
    y2 = np.linspace(0,4,100)
    y3 = np.linspace(4,8,100)
    # preparing light waves
    lx1,=ax.plot([], [], 'r')
    lx2,=ax.plot([], [], 'r')
    lx3,=ax.plot([], [], 'b')
    lx4,=ax.plot([], [], 'r')
    lx5,=ax.plot([], [], 'r')
    lx6,=ax.plot([], [], 'r')
    ly1,=ax.plot([], [], 'r')
    ly2,=ax.plot([], [], 'r')
    ly3,=ax.plot([], [], 'r')  
    
       
    # the interferometer paths Syntax: (xdata), (ydata) for the various points
    up = plt.Line2D((1,9),(4,4),color="whitesmoke")
    do = plt.Line2D((0,5),(0,0),color="whitesmoke")
    le = plt.Line2D((1,1),(0,4),color="whitesmoke")
    ri = plt.Line2D((5,5),(0,8),color="whitesmoke")
 

    #mirrors
    m1 = plt.Line2D((0.5,1.5),(3.5,4.5),color="black") # mirror at top left corner
    m2 = plt.Line2D((4.5,5.5),(-0.5,0.5),color="black") # mirror at bootom right corner

    # beam spitters  (For calcualtions, first beam splitter will have reflecting surface on the left side, Seond will have reflectivng surface on the right side)
    bs1 = plt.Line2D((0.5,1.5),(-0.5,0.5),color="grey",linestyle='dashed',lw=2.5)
    bs2 = plt.Line2D((4.5,5.5),(3.5,4.5),color="grey",linestyle='dashed',lw=2.5)

    
    # adding elements to the figures 
    # creating the glass slide in the lower arm of the Mach-Zehnder
    patch = patches.Rectangle((xglass,-0.625), width, 1.25, facecolor='white',edgecolor='black')
    glass=ax.add_patch(patch)
    ax.add_line(up)
    ax.add_line(do)
    ax.add_line(le)
    ax.add_line(ri)
    ax.add_line(m1)
    ax.add_line(m2)
    ax.add_line(bs1)
    ax.add_line(bs2)
    
    
    # creating incoming sine wave    
    lx1.set_data(x1, Amp*np.sin(k*(x1-c*t))*np.exp(-a*(x1-vg*t)**2))

    # creating the wave transmitted through the first beam splitter    
    lx2.set_data(x2, t1*Amp*np.sin(k*(x2-c*t))*np.exp(-a*(x2-vg*t)**2))

    # creating the wave travelling inside the glass slide    
    lx3.set_data(x3,t1*Amp*np.sin(eta*k*(x3-xglass-c*t/eta)+k*xglass)*np.exp(-a*(x3-vg*t+(eta-1)*(x3-xglass))**2))

    # creating the wave emerging from the glass slide and htiing mirror m2    
    lx4.set_data(x4,t1*Amp*np.sin(k*(x4-c*t) + (eta-1)*k*width)*np.exp(-a*(x4-vg*t+(eta-1)*width)**2))

    # creating the wave reflecting off the first beamsplitter (pi phase shift will occur on reflection here)    
    ly1.set_data(1 + r1*Amp*np.sin(k*(y1-0-c*t) + np.pi + k*1)*np.exp(-a*(1+y1-0-vg*t)**2),y1)

    # creating the wave reflecting off the mirror m2 (pi phase shift will occur on reflection here)    
    ly2.set_data(5 + t1*Amp*np.sin(k*(y1-0-c*t) + np.pi + k*5 + (eta-1)*k*width )*np.exp(-a*(5+(eta-1)*width+y2-0-vg*t)**2),y1)

    # creating the wave reflecting off the mirror m1 (pi phase shift will occur on reflection here)    
    lx5.set_data(x5,4+r1*Amp*np.sin(k*(x5-c*t) + k*(4-0) + 2*np.pi)*np.exp(-a*(4+x5-vg*t)**2))

    # creating the wave emerging on the horizontal path after the second beam splitter 
    #(pi phase shift occurs for the wave in the lower path that is reflected to the right)    
    lx6.set_data(x6,4 + Amp*interfere_amplitudes(r1*t2*np.exp(-a*(4+x6-vg*t)**2),k*9,t1*r2*np.exp(-a*(4+(eta-1)*width+x6-0-vg*t)**2),k*9 +(eta-1)*k*width)*np.sin(k*(x6-5-c*t)+interfere_phases(r1*t2*np.exp(-a*(4+x6-vg*t)**2),k*9,t1*r2*np.exp(-a*(4+(eta-1)*width+x6-0-vg*t)**2),k*9 +(eta-1)*k*width)))

    # creating the wave emerging on the vertical path after the second beam splitter 
    #(NO phase shift occurs when the wave moving right is reflected upwards by the beam splitter. 
    # This is because we had chosen the convention that the second beam splitter has mirrored surface on the right hand side)
    ly3.set_data(5+Amp*interfere_amplitudes(r1*r2*np.exp(-a*(5+y3-vg*t)**2),k*9,t1*t2*np.exp(-a*(5+(eta-1)*width+y3-0-vg*t)**2),k*9 +(eta-1)*k*width+np.pi)*np.sin(k*(y3-4-c*t)+interfere_phases(r1*r2*np.exp(-a*(5+y3-vg*t)**2),k*9,t1*t2*np.exp(-a*(5+(eta-1)*width+y3-0-vg*t)**2),k*9 +(eta-1)*k*width+np.pi)),y3)
    
    plt.axes(ax)
    # Axis scaling turned on so that aspect ratio is maintained
    plt.axis("scaled")
    # Axis turned off 
    plt.axis("off")
    plt.show()

style = {'description_width': 'initial'}    #to be abto to show full description of the sliders
@widgets.interact(a=widgets.FloatLogSlider(
    value=10,
    base=10,
    min=-10, # max exponent of base
    max=2, # min exponent of base
    step=0.2, # exponent step
    description='Compress wave packet',style=style
    ),vg_factor=widgets.FloatSlider(
    value=1,
    min=0,
    max=1, 
    step=0.001, 
    description='f (where group velocity = f x c)',style=style
    ),t=widgets.FloatSlider(
    value=1,
    min=0,
    max=100, 
    step=0.001, 
    description='time'
    ),c=(0,1,0.001),width=widgets.FloatSlider(
    value=0.78,
    min=0,
    max=1, 
    step=0.001, 
    description='width of the phase shifter',style=style
    ),xglass=widgets.FloatSlider(
    value=2.5,
    min=1.5,
    max=3.5, 
    step=0.001, 
    description='Position of the phase shifters',style=style
    ),eta=widgets.FloatSlider(
    value=3,
    min=1,
    max=3, 
    step=0.001, 
    description='Refractive index',style=style
    ),k=(1,30,0.1),Amp=(0.062,0.25,0.001),t1=widgets.FloatSlider(
    value=0.707,
    min=0.001,
    max=0.999, 
    step=0.001, 
    description='Transmittivity of first BS',style=style
    ),t2=widgets.FloatSlider(
    value=0.707,
    min=0.001,
    max=0.999, 
    step=0.001, 
    description='Transmittivity of second BS',style=style
    ))
def update(a=0.8,vg_factor=1,c=0.2,t=0,width = 0.78,xglass=2.5,eta=3,k=22.2,Amp=0.125,t1=0.707,t2=0.707):
    ax=plt.gca() #select axis object contining old figure
    #plt.cla() #clear old figure
    vg=vg_factor*c
    create_figure(a,vg,c,t,ax,width,xglass,eta,k,Amp,t1,t2)# create new figure
    




interactive(children=(FloatLogSlider(value=2.0, description='Compress wave packet', max=2.0, min=-10.0, step=0…